In [1]:
import numpy as np
import pandas as pd

import sys
import os
import random

from PIL import Image, ImageFont, ImageDraw
from PIL.ImageChops import add, subtract, multiply, difference, screen

import cv2

import matplotlib.pyplot as plt

https://pillow.readthedocs.io/en/stable/reference/Image.html

# OpenCV

http://www-cs.ccny.cuny.edu/~wolberg/capstone/opencv/LearningOpenCV.pdf

http://opencvexamples.blogspot.com/p/learning-opencv-functions-step-by-step.html

https://opencv.org/

https://github.com/opencv/opencv/tree/master/.github

https://pythonprogramming.net/corner-detection-python-opencv-tutorial/

## Image Overlays Using Bitwise Operations in OpenCV

In [2]:
home = os.environ['HOME']
#https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f

path = os.path.join(home, 'git', 'deep-food', 'gleb_google_images')
path

'/Users/iskriyanavasileva/git/deep-food/gleb_google_images'

In [3]:
path_comb = os.path.join(path, 'gleb_google_images' + '_comb')
path_comb

'/Users/iskriyanavasileva/git/deep-food/gleb_google_images/gleb_google_images_comb'

In [4]:
def load_bg_random(path, bg_folder='background'):
    """
    chooses a random background image
    """    
    #https://stackoverflow.com/questions/701402/best-way-to-choose-a-random-file-from-a-directory
    #https://www.techtrekking.com/how-to-randomly-select-files-from-folder-using-python/
    #https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randint.html
    bg_folder = os.path.join(path, bg_folder)
    bg_files = os.listdir(bg_folder)
    bg_files = [f for f in bg_files if not f[0] == '.']
    bg_index = random.randrange(0, len(bg_files))
    
    bg = os.path.join(path, bg_folder, bg_files[bg_index])
    
    background = cv2.imread(bg)
    rows_b, cols_b, channels_b = background.shape
    
    return background, rows_b, cols_b, channels_b 

In [1]:
def load_img_mask(path):
    """
    creates a mask of the ingredient
    creates a new combined image
    """        
    for root, dirs, files in os.walk(path):
            #https://stackabuse.com/python-check-if-string-contains-substring/
            #https://cyluun.github.io/blog/manipulating-python-oswalk
            #https://bugsdb.com/_en/debug/064b3876f2e6bc643f0553131e2776b4
            
            files = [f for f in files if not f[0] == '.' and f.find('background') == -1]
            for f in files:
                file_path = os.path.join(root, f)
                
                #https://www.geeksforgeeks.org/reading-images-in-python/
                ingredient = cv2.imread(file_path)
                
                #load random background
                background, rows_b, cols_b, channels_b = load_bg_random(path)
                
                # resize ingredient image to make sure it is always smaller than the background
                #resize (cols, rows)
                #https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
                #https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
                #https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
                #https://www.pyimagesearch.com/2014/01/20/basic-image-manipulations-in-python-and-opencv-resizing-scaling-rotating-and-cropping/
                
                #ASPECT RATIOS
                #https://support.squarespace.com/hc/en-us/articles/115008538927-Understanding-aspect-ratios#:~:text=An%20aspect%20ratio%20is%20a,and%20width%20are%20the%20same.
                #https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
                #https://stackoverflow.com/questions/2232742/does-python-pil-resize-maintain-the-aspect-ratio
                
                rows_i = int(rows_b/2)
                cols_i = int(cols_b/2)
                dim_res = (cols_i, rows_i)
                ingredient_res = cv2.resize(ingredient, dim_res)
                
                # Create a Region of Images (=ROI) in a randomised matter 
                rows, cols, channels = ingredient_res.shape
                
                #https://pynative.com/python-random-randrange/
                start_rows = np.random.randint(0, rows_b-rows)
                end_rows = start_rows + rows
                start_cols = np.random.randint(0, cols_b-cols)
                end_cols = start_cols + cols

                #https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_core/py_basic_ops/py_basic_ops.html
                #https://www.learnopencv.com/how-to-select-a-bounding-box-roi-in-opencv-cpp-python/
                roi = background[start_rows:end_rows, start_cols:end_cols]
                
                #parameters for rotation
                #M are the coordinates of the center, (cols/2, rows/2) is the center of rotation for the image
                #https://livecodestream.dev/post/2020-06-11-essential-opencv-functions-to-get-you-started-into-computer-vision/
                deg = np.random.randint(0, 360)
                M = cv2.getRotationMatrix2D((cols/2, rows/2), deg, 1)
                
                ingredient_rot = cv2.warpAffine(ingredient_res, M, (cols, rows))

                # Now create a mask of the ingredient and create its inverse mask also
                #https://techtutorialsx.com/2019/04/13/python-opencv-converting-image-to-black-and-white/#:~:text=Converting%20an%20image%20to%20black%20and%20white%20with%20OpenCV%20can,the%20value%200%20(black).
                #https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html
                img2gray = cv2.cvtColor(ingredient_res, cv2.COLOR_BGR2GRAY)

                # everything higher than the threshold (i.e. lighter), will become black
                # the rest (i.e. the darker pixels filled by the object), will become white
                # with THRESH_OTSU the threshold is adjustable according to the image
                # similar to the idea to take the mean value between black and white as a threshold
                # the GaussianBluring smooths the image to make the black & white more precise
                blur = cv2.GaussianBlur(img2gray, (5,5), 0)
                
                #https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html
                #https://stackoverflow.com/questions/22240220/thresholding-image-in-opencv-for-a-range-of-max-and-min-values
                #https://docs.opencv.org/3.4/db/d8e/tutorial_threshold.html
                #https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html
                ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                
                #MAIN LINKS MASKING
                #https://theailearner.com/2019/03/26/image-overlays-using-bitwise-operations-opencv-python/
                #https://towardsdatascience.com/weekend-project-detecting-solar-panels-from-satellite-imagery-f6f5d5e0da40
                #https://docs.opencv.org/master/d0/d86/tutorial_py_image_arithmetics.html
                #https://www.learnopencv.com/alpha-blending-using-opencv-cpp-python/
                #https://www.geeksforgeeks.org/arithmetic-operations-on-images-using-opencv-set-2-bitwise-operations-on-binary-images/
                #https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_core/py_image_arithmetics/py_image_arithmetics.html
                #https://note.nkmk.me/en/python-opencv-numpy-alpha-blend-mask/
                #https://medium.com/@vishwasu2001/basic-arithmetic-and-bitwise-operations-on-images-with-open-cv-part-4-87d04e2a42c
                #https://towardsdatascience.com/understanding-the-basics-of-digital-image-processing-and-computer-vision-using-opencv-8bc0e67243c8
                #https://towardsdatascience.com/practical-image-process-with-opencv-8405772c603e
                
                #https://scikit-image.org/docs/dev/user_guide/transforming_image_data.html
                #https://scikit-image.org/
                #https://stackoverflow.com/questions/10469235/opencv-apply-mask-to-a-color-image/38493075
                #https://towardsdatascience.com/extracting-faces-using-opencv-face-detection-neural-network-475c5cd0c260
                #https://towardsdatascience.com/facial-keypoints-detection-deep-learning-737547f73515
                #https://towardsdatascience.com/image-panorama-stitching-with-opencv-2402bde6b46c
                #https://datacarpentry.org/image-processing/04-drawing/
                #https://stackoverflow.com/questions/2169605/use-numpy-to-mask-an-image-with-a-pattern
                #https://likegeeks.com/python-image-processing/
                #https://ai.stackexchange.com/questions/13476/how-to-mask-an-image-using-numpy-opencv
                #https://campus.datacamp.com/courses/biomedical-image-analysis-in-python/masks-and-filters?ex=6
                
                #TO CHECK
                #https://www.tutorialfor.com/questions-99662.htm
                #https://towardsdatascience.com/practical-image-process-with-opencv-8405772c603e
                
                    
                mask_rot = cv2.warpAffine(mask, M, (cols, rows))
                
                #https://docs.opencv.org/4.1.2/d0/d86/tutorial_py_image_arithmetics.html                
                mask_inv = cv2.bitwise_not(mask_rot)

                # Black-out the area of the ingredient in ROI in the background image
                background_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)

                # Take only region of the ingredient from the ingredient image.
                ingredient_fg = cv2.bitwise_and(ingredient_rot, ingredient_rot, mask = mask_rot)

                # Put the ingredient in ROI and modify the main image
                dst = cv2.add(background_bg, ingredient_fg)

                #positions are randomised (see above)
                background[start_rows:end_rows, start_cols:end_cols] = dst
                
                img_comb = background 
                           
                # select only the filename: https://stackoverflow.com/questions/678236/how-to-get-the-filename-without-the-extension-from-a-path-in-python?rq=1
                filename = os.path.splitext(f)[0]
                file_path_comb = os.path.join(root, f'{filename}_comb.jpg')
                #https://www.geeksforgeeks.org/python-opencv-cv2-imwrite-method/
                cv2.imwrite(file_path_comb, img_comb)

In [6]:
load_img_mask(path)